# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install torch torchvision transformers pandas

In [ ]:
import os
from PIL import Image  # Ensure correct import of PIL.Image
import torchvision 
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from IPython.display import display, Image
from transformers import SwinConfig, SwinForImageClassification
from PIL import ImageFile
import torch.optim as optim
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split

In [ ]:
# Initialize an empty list to store the data
data = []

# Set the base directory where the images are stored
base_dir = '/kaggle/input/comprehensive-disaster-datasetcdd/Comprehensive Disaster Dataset(CDD)'

# Walk through the directory structure
for dirname, _, filenames in os.walk(base_dir):
    for filename in filenames:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            label = os.path.basename(dirname)
#             full_path = os.path.join(dirname, filename)
            data.append({'filename': filename, 'label': label})

# Create a DataFrame from the list
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Base directory where the images are stored
base_dir = '/kaggle/input/comprehensive-disaster-datasetcdd/Comprehensive Disaster Dataset(CDD)'

# Prepare a list to collect data
data = []

# Walk through the directory structure
for dirname, _, filenames in os.walk(base_dir):
    for filename in filenames:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            category = os.path.basename(dirname)  # The category is the name of the folder
            data.append({'Image ID': filename, 'Category': category})

# Create a DataFrame
df = pd.DataFrame(data)

# Display the first few rows to verify
df.head()
# Save the DataFrame to CSV if needed
df.to_csv('/kaggle/working/image_paths.csv', index=False)
df.head()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
import multiprocessing

multiprocessing.set_start_method('fork', force=True)


In [ ]:
class DisasterDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        
        # Create a mapping from string labels to integers
        self.label_mapping = {label: idx for idx, label in enumerate(dataframe['Category'].unique())}
        self.dataframe['label'] = dataframe['Category'].map(self.label_mapping)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx, 2]  # Changed to use the mapped integer label
        
        if self.transform:
            image = self.transform(image)
        
        # Convert label to tensor
        label = torch.tensor(label).long()
        
        return image, label
# Assuming df is your DataFrame
root_dir = '/kaggle/input/combind-dataset/Images Data'  # Update this to the directory containing your images

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
# Create the dataset
full_dataset = DisasterDataset(df, root_dir, transform=transform)

In [ ]:
# Split the dataset into train, validation, and test sets
train_indices, temp_indices = train_test_split(list(range(len(full_dataset))), test_size=0.2, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Create Subsets
train_subset = Subset(full_dataset, train_indices)
val_subset = Subset(full_dataset, val_indices)
test_subset = Subset(full_dataset, test_indices)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:

# Modify the SwinConfig for a smaller model
config = SwinConfig.from_pretrained(
    'microsoft/swin-tiny-patch4-window7-224',
    num_labels=len(full_dataset.label_mapping),
    hidden_dropout_prob=0.6,
    attention_probs_dropout_prob=0.6
)

# Adjust the number of layers (depths) in each stage
# Original depths for Swin-Tiny: [2, 2, 6, 2]
config.depths = [2, 2, 5, 2]  # Reduce the number of transformer layers

# Instantiate the model with the modified configuration
model = SwinForImageClassification(config)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
import os
from PIL import Image  # Ensure correct import of PIL.Image
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split


def imshow(img, title=None):
    img = img.numpy().transpose((1, 2, 0))  # convert from Tensor image
    mean = np.array([0.5, 0.5, 0.5])  # example mean for normalization used
    std = np.array([0.5, 0.5, 0.5])  # example std for normalization used
    img = std * img + mean  # denormalize
    img = np.clip(img, 0, 1)  # clip the image values to be between 0 and 1
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Print some debug information
print(f"Total dataset size: {len(full_dataset)}")
print(f"Training set size: {len(train_subset)}")
print(f"Validation set size: {len(val_subset)}")
print(f"Test set size: {len(test_subset)}")

# Verify DataLoader functionality
try:
    dataiter = iter(train_loader)
    images, labels = next(dataiter)
    print("Batch loaded successfully.")
    
    # Show images
    imshow(torchvision.utils.make_grid(images), title='Some Training Images')
except KeyError as e:
    print(f"KeyError encountered: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Function to visualize images
def imshow(img, title=None):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    if title is not None:
        plt.title(title)
    plt.show()

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = CrossEntropyLoss()

# Scheduler
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # Decay LR by a factor of 0.1 every 3 epochs

# Early Stopping Class (as defined above)
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Initialize early stopping
early_stopping = EarlyStopping(patience=5, min_delta=0.01)

# Lists to keep track of losses
train_losses, val_losses = [], []
num_epochs = 100  # Set a high number of epochs and rely on early stopping

for epoch in range(num_epochs):
    model.train()
    total_loss, total_val_loss = 0, 0
    
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Visualize the first batch of each epoch
        if i == 0:
            plt.figure(figsize=(10, 10))
            imshow(torchvision.utils.make_grid(images.cpu()), title=f'Epoch {epoch+1} - First batch')
            plt.show()
        
        optimizer.zero_grad()
        outputs = model(images)
        logits = outputs.logits  # Extract the logits from the output
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    train_losses.append(total_loss / len(train_loader))
    
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            logits = outputs.logits  # Extract the logits from the output
            val_loss = criterion(logits, labels)
            total_val_loss += val_loss.item()
    
    val_losses.append(total_val_loss / len(val_loader))
    print(f'Epoch {epoch+1}, Train Loss: {train_losses[-1]}, Val Loss: {val_losses[-1]}')

    scheduler.step()  # Adjust the learning rate based on the scheduler
    
    # Check early stopping
    if early_stopping(total_val_loss / len(val_loader)):
        print("Early stopping triggered")
        break

# Plot learning curves
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
label_mapping = full_dataset.label_mapping
index_to_class = {v: k for k, v in label_mapping.items()} # Reverse the mapping

In [ ]:
def denormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor


In [ ]:
# Define the test loop to calculate test accuracy
def test_model(model, test_loader, criterion, device):
    model.eval()
    total_test_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            logits = outputs.logits  # Extract the logits from the output
            loss = criterion(logits, labels)
            total_test_loss += loss.item()

            # Get the predicted class with the highest score
            _, predicted = torch.max(logits, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    average_test_loss = total_test_loss / len(test_loader)
    test_accuracy = correct_predictions / total_predictions

    print(f'Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')
    return average_test_loss, test_accuracy

# Calculate test loss and accuracy
test_loss, test_accuracy = test_model(model, test_loader, criterion, device)


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import torchvision

def imshow(img, title=None):
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    img = denormalize(img, mean, std)  # Denormalize the image
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Convert from Tensor image and transpose to match image format
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # Pause a bit so that plots are updated

# Modify the visualize_test_results function to display class names
def visualize_test_results(model, test_loader, device, num_images=20):
    model.eval()
    images_shown = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)

            # Move images and predictions to CPU for visualization
            images = images.cpu()
            predicted = predicted.cpu()
            labels = labels.cpu()

            for i in range(images.size(0)):
                if images_shown >= num_images:
                    break
                plt.figure(figsize=(6, 6))
                imshow(torchvision.utils.make_grid(images[i]),
                       title=f'True: {index_to_class[labels[i].item()]}, Pred: {index_to_class[predicted[i].item()]}')
                images_shown += 1

            if images_shown >= num_images:
                break

# Assuming you have already created the test_loader
visualize_test_results(model, test_loader, device)
